The Bloom_Filter functions takes our required input file and provides our desired output.

To use it, call the function with:
`lookup_count,FP_count=Bloom_Filter(inputfile,outputfile)`



Note: In place of **inputfile** you can place your input csv file.

**outputfile** will contain all the unique words of inputfile, with the duplicates removed. So thats our desired output.

We need to put an empty .csv file at the desired location beforehand for **outputfile**.



Also the function returns two output- **lookup_count** -> the number of lookups needed.

**FP_count** -> the number of false positive by our algorithm.

In [1]:
# Import required Libraries

import csv
import numpy as np
import xxhash
import mmh3

In [2]:
def Bloom_Filter(inp_filename,op_filename, exp_elem, hash_func_num, prob_false_pos = 0.8):

     
     
     
     lookup_count=0  # will store count of number of lookups
     FP_count=0 # will store count of number of False Positives


     
     # Take our desired expected no. of element and probability of false positives from the users
    #  exp_elem=float(input('What is the expected no. of Elements in our sample: '))
    #  prob_false_pos=float(input('What is the probability of false positive: '))



     # Calculating bit array side according to formula
     bit_arr_size=round(-1*((exp_elem*np.log(prob_false_pos))/(np.square(np.log(2)))))
     # Calculating no. of hash functions according to formula
    #  hash_func_num= round((bit_arr_size/exp_elem)*np.log(2))
    
     print("number of hash functions", hash_func_num)
     print("bit array size", bit_arr_size)


     # Generating the hash functions. Here we use a random function: (sum_of_ascii_values_of_word)%bit_arr_size+i , where i refers to the index from 0
    # to no.of hash function  . Suppose a particular bloom filter is 4 bit-array, and 2 hash functions so
    # Eg: H1-> ascii("Hello")%4+0
    #     H2 -> ascii("Hello")%4+1
     def filters_to_have():
            lst_fltr=[]
            for i in range(hash_func_num):
                # lst_fltr.append(xxhash.xxh32(word, seed=42).intdigest()%bit_arr_size+1)
                lst_fltr.append(mmh3.hash(word, seed=0)%bit_arr_size)
        # Note this function creates the required no. of hash functions with the above formula. The frmula is random and there is scope of improvement
        # If needed to add own formula, the user can delete one of the formulas from lst_fltr and add their own formula
            return lst_fltr



    # Create the bit array with the required size and fill them inititally with 0. We use Python list to create the bit array
     bloom_fltr=[0]*bit_arr_size

    
    # we will read each and every word from .csv file one-by-one and apply the bloom filter
     with open(inp_filename, 'r') as file:
         my_reader = csv.reader(file, delimiter=',')
         next(file)
         for row in my_reader:
              word=row[0]
              char_val_sum=sum([ord(z) for z in list(word)]) # getting sum of ascii vals of characters
              lst_fltr=filters_to_have() # generating the hash values for each word
              lst_fltr=[len(bloom_fltr)-1  if i>len(bloom_fltr)-1 else i for i in lst_fltr ] # the index/hash value should not exceed max size of bit array so clipping the ones which are exceeding

              # Bloom Filter below main logic to find out the words present

              # checking how many 0's are obtained in bit array for each hash values
              # suppose there are 4 hash values. If sum_all is 4 then all are 0's
              sum_all=sum([bloom_fltr[i]==0 for i in lst_fltr])
              # even a single 0 in bit array for any of the hash value will prove the word is not present/non-duplicate so we will add them in output file
              if sum_all>0:
                    with open(op_filename, 'a', newline='') as csvfile:
                          spamwriter1 = csv.writer(csvfile, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                          spamwriter1.writerow([word])
                    for ij in lst_fltr:
                        bloom_fltr[ij]=1
              # if sum_all=0 then we need to lookup our outputfile to confirm that word is really present or it's a false positive
              else:
                    # lookup count increases by 1 since we are perfoming lookup
                    lookup_count+=1
                    # logic for checking whether the word is indeed present in the outputfile
                    cc=0
                    with open(op_filename, 'r') as file:
                            my_reader = csv.reader(file, delimiter=',')
                            # next(file)
                            for row1 in my_reader:
                              word1=row1[0]

                              if word==word1:
                                  cc+=1

                    if cc>0:
                        pass
                    else:
                           # if the case in false positive(FP) increment the FP_count by 1
                           FP_count+=1
                           # since the case is FP so originally the word is not present in the outputfile, so add it there
                           with open(op_filename, 'a', newline='') as csvfile:
                                 spamwriter3 = csv.writer(csvfile, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                 spamwriter3.writerow([word])

                    
    


     
     
     # Printing the lookup counts and false positive values
     print('Number of False Positives: ', FP_count)
     print('Number of Lookup Count: ', lookup_count)

     # Returning the values of lookup counts and false positive
     return [lookup_count,FP_count]

In [15]:
import pandas as pd
df = pd.read_csv('streaming_words.csv')
df = df.sample(frac=0.4).reset_index(drop=True)
df = df.loc[:50000]
df.to_csv("sample.csv", index = False)

In [16]:
import time
for n in range(15000, 50000, 5000):
    for i in range(1, 5, 1):
        start_time = time.time()
        with open("word_drop{}_{}.csv".format(n,i), "a") as file:
            pass
        Bloom_Filter("sample.csv", "word_drop{}_{}.csv".format(n,i), n, i, 0.8)
        print("Time taken for n = {} and k = {} is {}".format(n, i, time.time()-start_time))

number of hash functions 1
bit array size 6967


KeyboardInterrupt: 

In [7]:
1694742260.6094174 - 1694740918.500419

1342.1089985370636

In [8]:
1694742260.6094515  - 1694743273.0002966

-1012.3908450603485

In [10]:
1694743273.0003104 - 1694744217.9540553

-944.9537448883057

In [11]:
1694744217.9540713 - 1694745140.208093

-922.2540216445923

In [12]:
1694745140.2081053 -  1694746052.281659

-912.0735535621643

In [28]:
#n = 333425, p = 0.1
lookup_count,FP_count=Bloom_Filter('streaming_words.csv','word_drop.csv')

print('------------')
print('LOOKUP COUNT ',lookup_count)
print('------------')
print('FP_COUNT ',FP_count)

number of hash functions 3
bit array size 1597949
Number of False Positives:  32725
Number of Lookup Count:  32818
------------
LOOKUP COUNT  32818
------------
FP_COUNT  32725


Notice the difference in outputs in thw two above for different choice of expected no. of elements and FP Probability.

The unique words are saved in **word_drop.csv**

In [31]:
#n = 333425, p = 0.05
lookup_count,FP_count=Bloom_Filter('streaming_words.csv','word_drop_1.csv')

print('------------')
print('LOOKUP COUNT ',lookup_count)
print('------------')
print('FP_COUNT ',FP_count)

number of hash functions 4
bit array size 2078980
Number of False Positives:  25290
Number of Lookup Count:  25383
------------
LOOKUP COUNT  25383
------------
FP_COUNT  25290


In [ ]:
#n = 333425, p = 0.01
lookup_count,FP_count=Bloom_Filter('streaming_words.csv','word_drop_2.csv')

print('------------')
print('LOOKUP COUNT ',lookup_count)
print('------------')
print('FP_COUNT ',FP_count)

In [ ]:
#n = 333425, p = 0.005
lookup_count,FP_count=Bloom_Filter('streaming_words.csv','word_drop_2.csv')

print('------------')
print('LOOKUP COUNT ',lookup_count)
print('------------')
print('FP_COUNT ',FP_count)